In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import winsound
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

In [2]:
raw_df = pd.read_csv('data\M1_2019.csv')

#raw_df['tick'] = (raw_df['High'] - raw_df['Low'])/2
raw_df['tick'] = raw_df['Close']

df = raw_df[['TS', 'tick']]
print(f'{len(df)} records in df')
df.head()

372530 records in df


,TS,tick
0,2019-01-01 17:02,1.14598
1,2019-01-01 17:03,1.14607
2,2019-01-01 17:04,1.14606
3,2019-01-01 17:05,1.14621
4,2019-01-01 17:06,1.14665


In [3]:
min_bar = 5
pip_diff = 0.0001
rsi_window = 14
window_s_ma = 10
window_l_ma = 100

In [4]:
df = df.iloc[::min_bar]
print(f'{len(df)} records under {min_bar} min_bar')

df['diff'] = df['tick'].diff()

df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])

df['avg_gain'] = df['gain'].rolling(window=rsi_window).mean()
df['avg_loss'] = df['loss'].rolling(window=rsi_window).mean()

df['rs'] = df['avg_gain']/df['avg_loss']
df['rsi'] = 100 - (100 / (df['rs'] + 1))


df['ssma'] = df['tick'].rolling(window=window_s_ma).mean()
df['lsma'] = df['tick'].rolling(window=window_l_ma).mean()

df['sema'] = df['tick'].ewm(span=window_s_ma).mean()
df['lema'] = df['tick'].ewm(span=window_l_ma).mean()

df['target'] = 'same'
df['target'].loc[df['tick'].shift(-1) - df['tick'] > pip_diff] = 'increase'
df['target'].loc[df['tick'].shift(-1) - df['tick'] < -pip_diff] = 'decrease'


df = df.dropna()
df = df.reset_index(drop=True)

74506 records under 5 min_bar


In [13]:
print(df['target'].value_counts())
print('--------------------------')
print(df['target'].value_counts(normalize=True))

same        40565
increase    17038
decrease    16804
Name: target, dtype: int64
--------------------------
same        0.545177
increase    0.228984
decrease    0.225839
Name: target, dtype: float64


In [9]:
del df['TS']
df.to_csv('data/tab_df.csv', index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

In [10]:
df.head(20)

,tick,diff,gain,loss,avg_gain,avg_loss,rs,rsi,ssma,lsma,sema,lema,target
0,1.14907,-0.00012,0.00000,0.00012,0.000326,0.000021,15.724138,94.020619,1.147613,1.145565,1.147735,1.145594,same
1,1.14917,0.00010,0.00010,0.00000,0.000333,0.000016,20.260870,95.296524,1.147931,1.145597,1.147996,1.145676,increase
2,1.14960,0.00043,0.00043,0.00000,0.000352,0.000016,21.434783,95.542636,1.148222,1.145633,1.148287,1.145765,decrease
3,1.14912,-0.00048,0.00000,0.00048,0.000324,0.000051,6.394366,86.476190,1.148446,1.145663,1.148439,1.145841,same
4,1.14917,0.00005,0.00005,0.00000,0.000294,0.000051,5.788732,85.269710,1.148670,1.145694,1.148572,1.145916,decrease
5,1.14885,-0.00032,0.00000,0.00032,0.000278,0.000074,3.776699,79.065041,1.148853,1.145722,1.148622,1.145983,same
6,1.14884,-0.00001,0.00000,0.00001,0.000228,0.000074,3.067308,75.413712,1.148988,1.145750,1.148662,1.146047,decrease
7,1.14870,-0.00014,0.00000,0.00014,0.000214,0.000084,2.542373,71.770335,1.149009,1.145776,1.148669,1.146106,decrease
8,1.14829,-0.00041,0.00000,0.00041,0.000211,0.000114,1.855346,64.977974,1.149000,1.145798,1.148600,1.146155,same
9,1.14829,0.00000,0.00000,0.00000,0.000204,0.000114,1.798742,64.269663,1.148910,1.145819,1.148544,1.146203,increase
